<a href="https://colab.research.google.com/github/ShinAsakawa/ShinAsakawa.github.io/blob/master/2022notebooks/2022_0112task_summary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- date: 2022_0112
- source url: https://colab.research.google.com/github/huggingface/notebooks/blob/master/transformers_doc/task_summary.ipynb
- filename: 2022_0112task_summary.ipynb


In [1]:
import platform

isColab = platform.system() == 'Linux'
if isColab:
    # Transformers installation
    !pip install transformers datasets > /dev/null 2>&1 
    # To install from source instead of the last release, comment the command above and uncomment the following one.
    #! pip install git+https://github.com/huggingface/transformers.git

In [2]:
if isColab:
    # MeCab, fugashi, ipadic のインストール
    !apt install aptitude swig > /dev/null 2>&1
    !aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y > /dev/null 2>&1
    !pip install mecab-python3 > /dev/null 2>&1
    !git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git > /dev/null 2>&1
    !echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n -a > /dev/null 2>&1
    
    import subprocess
    cmd='echo `mecab-config --dicdir`\"/mecab-ipadic-neologd\"'
    path_neologd = (subprocess.Popen(cmd, stdout=subprocess.PIPE,
                                     shell=True).communicate()[0]).decode('utf-8')

    !pip install 'fugashi[unidic]' > /dev/null 2>&1
    !python -m unidic download > /dev/null 2>&1
    !pip install ipadic > /dev/null 2>&1    

# 課題のまとめ
<!-- # Summary of the tasks -->

このページでは，ライブラリを使用する際に最も頻用されるユースケースを紹介します。
用意されているモデルは様々な構成が可能で，使用例も多岐にわたります。
ここでは，最も簡単なものを紹介し，質問応答，系列分類，名前付き実体認識 (NER) などの課題での使用を紹介します。
<!-- This page shows the most frequent use-cases when using the library. 
The models available allow for many different configurations and a great versatility in use-cases. 
The most simple ones are presented here, showcasing usage for tasks such as question answering, sequence classification, named entity recognition and others. -->

これらの例では，auto-models を活用しています。
Auto-models は，与えられたチェックポイントに従ってモデルを実体化し，正しいモデルアーキテクチャを自動的に選択するクラスです。
詳細は [AutoModel](https://huggingface.co/docs/transformers/master/en/model_doc/auto#transformers.AutoModel) の文書をご覧ください。
コードをより具体的に変更したり，特定のユースケースに合わせて自由にアレンジしてください。
<!-- These examples leverage auto-models, which are classes that will instantiate a model according to a given checkpoint, automatically selecting the correct model architecture. 
Please check the [AutoModel](https://huggingface.co/docs/transformers/master/en/model_doc/auto#transformers.AutoModel) documentation for more information. 
Feel free to modify the code to be more specific and adapt it to your specific use-case. -->

ある課題でモデルが十分な性能を発揮するためには，その課題に対応するチェックポイントからモデルを読み込む必要があります。
これらのチェックポイントは，通常，大規模なデータコーパスで事前訓練され，特定課題で微調整されています。
これは次のような意味を持ちます。
<!-- In order for a model to perform well on a task, it must be loaded from a checkpoint corresponding to that task. 
These checkpoints are usually pre-trained on a large corpus of data and fine-tuned on a specific task. 
This means the following: -->

- すべてのモデルがすべての課題で微調整されたわけではありません。
特定の課題でモデルを微調整したい場合は [examples](https://github.com/huggingface/transformers/tree/master/examples) ディレクトリにある*run_$TASK.py* スクリプトの 1 つを活用できます。
- 微調整済モデルは，特定のデータセットで微調整されています。
このデータセットは，あなたのユースケースやドメインと重なる場合もあれば，そうでない場合もあります。
前述のように [examples](https://github.com/huggingface/transformers/tree/master/examples) のスクリプトを利用してモデルを微調整することもできますし，独自の訓練クリプトを作成することもできます。

<!-- - Not all models were fine-tuned on all tasks. 
If you want to fine-tune a model on a specific task, you can leverage one of the *run_$TASK.py* scripts in the [examples](https://github.com/huggingface/transformers/tree/master/examples) directory.
- Fine-tuned models were fine-tuned on a specific dataset. 
This dataset may or may not overlap with your use-case and domain. 
As mentioned previously, you may leverage the [examples](https://github.com/huggingface/transformers/tree/master/examples) scripts to fine-tune your model, or you may create your own training script. -->

課題の推論を行うために，ライブラリにはいくつかの機構が用意されています。
<!-- In order to do an inference on a task, several mechanisms are made available by the library:-->

- パイプライン：非常に使いやすい抽象化で，わずか 2 行のコードで済みます。
- モデルの直接利用。トークン化器 (PyTorch/TensorFlow) に直接アクセスし，推論能力をフルに発揮することで，抽象化は少ないが，より高い柔軟性とパワーを発揮する。

<!-- - Pipelines: very easy-to-use abstractions, which require as little as two lines of code.
- Direct model use: Less abstractions, but more flexibility and power via a direct access to a tokenizer (PyTorch/TensorFlow) and full inference capacity. -->

両アプローチがここで紹介されています。

<!-- Both approaches are showcased here. -->

<Tip>

ここで紹介する全課題は，特定課題で微調整された事前訓練済チェックポイントを利用しています。
特定の課題で微調整されていないチェックポイントをロードすると，ベースの transformer 層のみがロードされ，課題で使用される追加ヘッドはロードされず，そのヘッドの重み係数はランダムに初期化されます。
<!-- All tasks presented here leverage pre-trained checkpoints that were fine-tuned on specific tasks. 
Loading a checkpoint that was not fine-tuned on a specific task would load only the base transformer layers and not the additional head that is used for the task, initializing the weights of that head randomly.-->

これにより，ランダムな出力が得られます。
<!--This would produce random output. -->

</Tip>

## 1. 系列分類
<!-- ## Sequence Classification -->

系列分類とは，与えられた数のクラスに従って系列を分類する課題です。
系列分類の例としては，GLUE データセットがありますが，これはすべてこの課題に基づいています。
GLUE の系列分類課題でモデルを微調整したい場合は，[run_glue.py](https://github.com/huggingface/transformers/tree/master/examples/pytorch/text-classification/run_glue.py)，[run_tf_glue.py](https://github.com/huggingface/transformers/tree/master/examples/tensorflow/text-classification/run_tf_glue.py)，[run_tf_text_classification.py](https://github.com/huggingface/transformers/tree/master/examples/tensorflow/text-classification/run_tf_text_classification.py)，[run_xnli.py](https://github.com/huggingface/transformers/tree/master/examples/pytorch/text-classification/run_xnli.py) のスクリプトを活用できます。
<!-- Sequence classification is the task of classifying sequences according to a given number of classes. 
An example of sequence classification is the GLUE dataset, which is entirely based on that task. 
If you would like to fine-tune a model on a GLUE sequence classification task, you may leverage the [run_glue.py](https://github.com/huggingface/transformers/tree/master/examples/pytorch/text-classification/run_glue.py), [run_tf_glue.py](https://github.com/huggingface/transformers/tree/master/examples/tensorflow/text-classification/run_tf_glue.py), [run_tf_text_classification.py](https://github.com/huggingface/transformers/tree/master/examples/tensorflow/text-classification/run_tf_text_classification.py) or [run_xnli.py](https://github.com/huggingface/transformers/tree/master/examples/pytorch/text-classification/run_xnli.py) scripts.-->

ここでは，パイプラインを使用して，系列が陽性 (ポジティブ) か陰性 (ネガティブ) かを識別する極性分析の例を紹介します。
GLUE 課題である sst2 の微調整されたモデルを利用しています。
<!-- Here is an example of using pipelines to do sentiment analysis: identifying if a sequence is positive or negative. 
It leverages a fine-tuned model on sst2, which is a GLUE task. -->

これは，以下のように得点と一緒にラベル (POSITIVE または NEGATIVE) を返します。
<!-- This returns a label ("POSITIVE" or "NEGATIVE") alongside a score, as follows: -->

In [ ]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")
result = classifier("I hate you")[0]
print(f"label: {result['label']}, with score: {round(result['score'], 4)}")


classifier_ja = pipeline("sentiment-analysis", model="daigo/bert-base-japanese-sentiment",tokenizer="daigo/bert-base-japanese-sentiment")
result_ja = classifier_ja("お前なんかきらいだ")[0]
print(f"ラベルl: {result_ja['label']}, 得点: {round(result_ja['score'], 4)}")

In [ ]:
result = classifier("I love you")[0]
print(f"label: {result['label']}, with score: {round(result['score'], 4)}")

# 日本語
result_ja = classifier_ja("お前が好きだ")[0]
print(f"ラベルl: {result_ja['label']}, 得点: {round(result_ja['score'], 4)}")

ここでは 2 つの配列が互いに言い換えられているかどうかを判断するために，モデルを使用して配列分類を行う例を示します。
処理は以下の通りです。
<!-- Here is an example of doing a sequence classification using a model to determine if two sequences are paraphrases of each other. 
The process is the following:-->

1. チェックポイント名からトークン化器とモデルを実体化する
モデルは，BERT モデルとして識別され，チェックポイントに保存されている重みでロードされます
2. 2 つの文から，正しいモデル固有のセパレータ，トークンタイプ ID，および注意マスク (トークン化器によって自動的に作成される) を使用して系列を構築します
3. この系列をモデルに通して，利用可能な 2 つのクラスのいずれかに分類する。0 (言い換えではない) と 1 (パラフレーズである) です
4. クラスの確率を得るために，結果のソフトマックスを計算します
5. 結果を印刷します

<!-- 
1. Instantiate a tokenizer and a model from the checkpoint name. 
The model is identified as a BERT model and loads it with the weights stored in the checkpoint.
2. Build a sequence from the two sentences, with the correct model-specific separators, token type ids and attention masks (which will be created automatically by the tokenizer).
3. Pass this sequence through the model so that it is classified in one of the two available classes: 0 (not a paraphrase) and 1 (is a paraphrase).
4. Compute the softmax of the result to get probabilities over the classes.
5. Print the results. -->

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased-finetuned-mrpc")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased-finetuned-mrpc")

classes = ["not paraphrase", "is paraphrase"]

sequence_0 = "The company HuggingFace is based in New York City"
sequence_1 = "Apples are especially bad for your health"
sequence_2 = "HuggingFace's headquarters are situated in Manhattan"

# トークン化器は，モデル固有のセパレータ (例 <CLS> や <SEP>) やトークンを自動的に系列に追加し，注意マスクを計算します。
# The tokenizer will automatically add any model specific separators (i.e. <CLS> and <SEP>) and tokens to the sequence, as well as compute the attention masks.
paraphrase = tokenizer(sequence_0, sequence_2, return_tensors="pt")
not_paraphrase = tokenizer(sequence_0, sequence_1, return_tensors="pt")

paraphrase_classification_logits = model(**paraphrase).logits
not_paraphrase_classification_logits = model(**not_paraphrase).logits

paraphrase_results = torch.softmax(paraphrase_classification_logits, dim=1).tolist()[0]
not_paraphrase_results = torch.softmax(not_paraphrase_classification_logits, dim=1).tolist()[0]

# 言い換えか？
# Should be paraphrase
for i in range(len(classes)):
    print(f"{classes[i]}: {int(round(paraphrase_results[i] * 100))}%")

In [ ]:
# 直上セルの日本語バージョン
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

model_name_ja = 'cl-tohoku/bert-base-japanese'
#model_name_ja = 'sonoisa/t5-base-japanese'
tknz_ja = AutoTokenizer.from_pretrained(model_name_ja)
model_ja = AutoModelForSequenceClassification.from_pretrained(model_name_ja)

classes = ["not paraphrase", "is paraphrase"]

sequence_0_ja = "東京女子大学は杉並区にある大学です"
sequence_1_ja = "東京大学は異世界に転生しました"
sequence_2_ja = "西荻窪駅には東京女子大学があります"

# トークン化器は，モデル固有のセパレータ (例 <CLS> や <SEP>) やトークンを自動的に系列に追加し，注意マスクを計算します。
# The tokenizer will automatically add any model specific separators (i.e. <CLS> and <SEP>) and tokens to the sequence, as well as compute the attention masks.
paraphrase_ja = tknz_ja(sequence_0_ja, sequence_2_ja, return_tensors="pt")
not_paraphrase_ja = tknz_ja(sequence_0_ja, sequence_1_ja, return_tensors="pt")

paraphrase_classification_logits_ja = model_ja(**paraphrase_ja).logits
not_paraphrase_classification_logits_ja = model_ja(**not_paraphrase_ja).logits

paraphrase_results_ja = torch.softmax(paraphrase_classification_logits_ja, dim=1).tolist()[0]
not_paraphrase_results_ja = torch.softmax(not_paraphrase_classification_logits_ja, dim=1).tolist()[0]

# 言い換えか？
# Should be paraphrase
for i in range(len(classes)):
    print(f"{classes[i]}: {int(round(paraphrase_results_ja[i] * 100))}%")

In [ ]:
# Should not be paraphrase
for i in range(len(classes)):
    print(f"{classes[i]}: {int(round(not_paraphrase_results[i] * 100))}%")

# 言い換えではない？ Should not be paraphrase
for i in range(len(classes)):
    print(f"{classes[i]}: {int(round(not_paraphrase_results_ja[i] * 100))}%")    

In [ ]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
import tensorflow as tf

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased-finetuned-mrpc")
model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-cased-finetuned-mrpc")

classes = ["not paraphrase", "is paraphrase"]

sequence_0 = "The company HuggingFace is based in New York City"
sequence_1 = "Apples are especially bad for your health"
sequence_2 = "HuggingFace's headquarters are situated in Manhattan"

# The tokenizer will automatically add any model specific separators (i.e. <CLS> and <SEP>) and tokens to
# the sequence, as well as compute the attention masks.
paraphrase = tokenizer(sequence_0, sequence_2, return_tensors="tf")
not_paraphrase = tokenizer(sequence_0, sequence_1, return_tensors="tf")

paraphrase_classification_logits = model(paraphrase).logits
not_paraphrase_classification_logits = model(not_paraphrase).logits

paraphrase_results = tf.nn.softmax(paraphrase_classification_logits, axis=1).numpy()[0]
not_paraphrase_results = tf.nn.softmax(not_paraphrase_classification_logits, axis=1).numpy()[0]

# Should be paraphrase
for i in range(len(classes)):
    print(f"{classes[i]}: {int(round(paraphrase_results[i] * 100))}%")

In [ ]:
# Should not be paraphrase
for i in range(len(classes)):
    print(f"{classes[i]}: {int(round(not_paraphrase_results[i] * 100))}%")

## 2. 抽出型質問応答
<!-- ## Extractive Question Answering -->

抽出型質問応答は，質問が与えられたテキストから答えを抽出する課題です。
質問応答データセットの例としては SQuAD データセットがあり，この課題に完全に基づいています。
SQuAD 課題でモデルを微調整したい場合は [run_qa.py](https://github.com/huggingface/transformers/tree/master/examples/pytorch/question-answering/run_qa.py) と [run_tf_squad.py](https://github.com/huggingface/transformers/tree/master/examples/tensorflow/question-answering/run_tf_squad.py) のスクリプトを活用してください。
<!-- Extractive Question Answering is the task of extracting an answer from a text given a question. 
An example of a question answering dataset is the SQuAD dataset, which is entirely based on that task. 
If you would like to fine-tune a model on a SQuAD task, you may leverage the [run_qa.py](https://github.com/huggingface/transformers/tree/master/examples/pytorch/question-answering/run_qa.py) and [run_tf_squad.py](https://github.com/huggingface/transformers/tree/master/examples/tensorflow/question-answering/run_tf_squad.py) scripts.-->

ここでは，パイプラインを使って質問応答を行う例を紹介します。
質問が与えられたテキストから答えを抽出します。
この例では SQuAD の微調整されたモデルを利用しています。
<!-- Here is an example of using pipelines to do question answering: extracting an answer from a text given a question. 
It leverages a fine-tuned model on SQuAD. -->

In [ ]:
from transformers import pipeline

question_answerer = pipeline("question-answering")

context = r"""
Extractive Question Answering is the task of extracting an answer from a text given a question. An example of a
question answering dataset is the SQuAD dataset, which is entirely based on that task. If you would like to fine-tune
a model on a SQuAD task, you may leverage the examples/pytorch/question-answering/run_squad.py script.
"""

これにより，テキストから抽出された回答，信頼度得点，および抽出された回答のテキスト中の位置である start と end の値が返されます。
<!-- This returns an answer extracted from the text, a confidence score, alongside "start" and "end" values, which are the positions of the extracted answer in the text. -->

In [11]:
result = question_answerer(question="What is extractive question answering?", context=context)
print(
    f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}"
)

Answer: 'the task of extracting an answer from a text given a question', score: 0.6177, start: 34, end: 95


In [12]:
result = question_answerer(question="What is a good example of a question answering dataset?", context=context)
print(
    f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}"
)

Answer: 'SQuAD dataset', score: 0.5152, start: 147, end: 160


In [13]:
# 直上 3 セルの日本語版
from transformers import pipeline

model_name_ja = 'cl-tohoku/bert-base-japanese'
question_answerer_ja = pipeline("question-answering", model=model_name_ja, tokenizer=model_name_ja)

# context = r"""
# Extractive Question Answering is the task of extracting an answer from a text given a question. 
# An example of a question answering dataset is the SQuAD dataset, which is entirely based on that task. 
# If you would like to fine-tune a model on a SQuAD task, you may leverage the examples/pytorch/question-answering/run_squad.py script.
# """

context_ja = r"""
抽出型質問応答とは，質問が与えられたテキストから答えを抽出する課題です。
質問応答データセットの例としては，SQuAD データセットがあり，この課題に完全に基づいています。
SQuAD 課題でモデルを微調整したい場合は，examples/pytorch/question-answering/run_squad.py スクリプトをご利用ください。
"""

result_ja = question_answerer_ja(question="抽出型質問応答とは何か？", context=context_ja)
print(
    f"Answer: '{result_ja['answer']}', 得点: {round(result_ja['score'], 4)}, start: {result_ja['start']}, end: {result_ja['end']}"
)

result_ja = question_answerer_ja(question="質問応答データセットの良い例は？", context=context_ja)
print(
    f"Answer: '{result_ja['answer']}', score: {round(result_ja['score'], 4)}, start: {result_ja['start']}, end: {result_ja['end']}"
)

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at cl-tohok

Answer: '抽出型質問応答とは，質問が与えられたテキストから答えを抽出する課題です。', 得点: 0.0002, start: 1, end: 37
Answer: '抽出型質問応答とは，質問が与えられたテキストから答えを抽出する課題です。', score: 0.0002, start: 1, end: 37


ここでは，モデルとトークン化器を使った質問応答の例を紹介します。
処理は以下の通りです。
<!-- Here is an example of question answering using a model and a tokenizer. 
The process is the following:-->

1. チェックポイント名からトークン化器とモデルを実体化する。モデルは BERT モデルとして識別され，チェックポイントに格納されている重みでロードされます。
2. テキストといくつかの質問を定義する。
3. 質問を繰り返し処理し，テキストと現在の質問から，モデル固有の正しいセパレータトー クン型 ID，注意マスクを使用して，系列を構築します。
4. この系列をモデルに通します。
これにより，開始位置と終了位置の両方について，系列トークン (質問とテキスト) 全体の得点の範囲が出力されます。
5. 結果のソフトマックスを計算して，トークンの確率を得る。
6. 識別された開始値と停止値からトークンを取得し，それらのトークンを文字列に変換します。
7. 結果を印刷します。

<!--
1. Instantiate a tokenizer and a model from the checkpoint name. The model is identified as a BERT model and loads it with the weights stored in the checkpoint.
2. Define a text and a few questions.
3. Iterate over the questions and build a sequence from the text and the current question, with the correct model-specific separators token type ids and attention masks.
4. Pass this sequence through the model. 
This outputs a range of scores across the entire sequence tokens (question and text), for both the start and end positions.
5. Compute the softmax of the result to get probabilities over the tokens.
6. Fetch the tokens from the identified start and stop values, convert those tokens to a string.
7. Print the results. -->

In [14]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch

tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
model = AutoModelForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

text = r"""
🤗 Transformers (formerly known as pytorch-transformers and pytorch-pretrained-bert) provides general-purpose
architectures (BERT, GPT-2, RoBERTa, XLM, DistilBert, XLNet…) for Natural Language Understanding (NLU) and Natural
Language Generation (NLG) with over 32+ pretrained models in 100+ languages and deep interoperability between
TensorFlow 2.0 and PyTorch.
"""

questions = [
    "How many pretrained models are available in 🤗 Transformers?",
    "What does 🤗 Transformers provide?",
    "🤗 Transformers provides interoperability between which frameworks?",
]

for question in questions:
    inputs = tokenizer(question, text, add_special_tokens=True, return_tensors="pt")
    input_ids = inputs["input_ids"].tolist()[0]

    outputs = model(**inputs)
    answer_start_scores = outputs.start_logits
    answer_end_scores = outputs.end_logits

    # Get the most likely beginning of answer with the argmax of the score
    answer_start = torch.argmax(answer_start_scores)
    # Get the most likely end of answer with the argmax of the score
    answer_end = torch.argmax(answer_end_scores) + 1

    answer = tokenizer.convert_tokens_to_string(
        tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end])
    )

    print(f"Question: {question}")
    print(f"Answer: {answer}")

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/443 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

Question: How many pretrained models are available in 🤗 Transformers?
Answer: over 32 +
Question: What does 🤗 Transformers provide?
Answer: general - purpose architectures
Question: 🤗 Transformers provides interoperability between which frameworks?
Answer: tensorflow 2. 0 and pytorch


In [ ]:
# Tensorflow 版なので全てコメントアウトした

# from transformers import AutoTokenizer, TFAutoModelForQuestionAnswering
# import tensorflow as tf

# tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
# model = TFAutoModelForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

# text = r"""
# 🤗 Transformers (formerly known as pytorch-transformers and pytorch-pretrained-bert) provides general-purpose
# architectures (BERT, GPT-2, RoBERTa, XLM, DistilBert, XLNet…) for Natural Language Understanding (NLU) and Natural
# Language Generation (NLG) with over 32+ pretrained models in 100+ languages and deep interoperability between
# TensorFlow 2.0 and PyTorch.
# """

# questions = [
#     "How many pretrained models are available in 🤗 Transformers?",
#     "What does 🤗 Transformers provide?",
#     "🤗 Transformers provides interoperability between which frameworks?",
# ]

# for question in questions:
#     inputs = tokenizer(question, text, add_special_tokens=True, return_tensors="tf")
#     input_ids = inputs["input_ids"].numpy()[0]

#     outputs = model(inputs)
#     answer_start_scores = outputs.start_logits
#     answer_end_scores = outputs.end_logits

#     # Get the most likely beginning of answer with the argmax of the score
#     answer_start = tf.argmax(answer_start_scores, axis=1).numpy()[0]
#     # Get the most likely end of answer with the argmax of the score
#     answer_end = tf.argmax(answer_end_scores, axis=1).numpy()[0] + 1

#     answer = tokenizer.convert_tokens_to_string(
#         tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end])
#     )

#     print(f"Question: {question}")
#     print(f"Answer: {answer}")

Question: How many pretrained models are available in 🤗 Transformers?
Answer: over 32 +
Question: What does 🤗 Transformers provide?
Answer: general - purpose architectures
Question: 🤗 Transformers provides interoperability between which frameworks?
Answer: tensorflow 2. 0 and pytorch

## 3. 言語モデル 
<!-- ## Language Modeling -->

言語モデルとは，モデルをコーパスに適合させる作業であり，ドメイン固有のものである可能性があります。
一般的な transfomrer ベースのモデルはすべて，
言語モデルの変形を用いて学習されています。
例えば BERT はマスク化言語モデルを，GPT-2 は因果言語モデルを用いています。
<!-- Language modeling is the task of fitting a model to a corpus, which can be domain specific. 
All popular transformer-based models are trained using a variant of language modeling, e.g. BERT with masked language modeling, GPT-2 with causal language modeling.-->

例えば，非常に大きなコーパスで学習された言語モデルを使用して，ニュースデータセットや科学論文で微調整することができます。
例えば [LysandreJik/arxiv-nlp](https://huggingface.co/lysandre/arxiv-nlp) などです。
<!-- Language modeling can be useful outside of pretraining as well, for example to shift the model distribution to be domain-specific: using a language model trained over a very large corpus, and then fine-tuning it to a news dataset or on scientific papers e.g. [LysandreJik/arxiv-nlp](https://huggingface.co/lysandre/arxiv-nlp). -->

### 3.1 マスク化言語モデル
<!-- ### Masked Language Modeling -->

マスク化言語モデルとは，一連のトークンをマスクトークンでマスクし，そのマスクを適切なトークンで埋めるようモデルに促す課題です。
これにより，モデルは右の文脈 (マスクの右にあるトークン) と左の文脈 (マスクの左にあるトークン) の両方に注意を払うことができます。
このようなトレーニングは SQuAD (質問応答 [Lewis, Lui, Goyal et al.](https://arxiv.org/abs/1910.13461) の 4.2 節参照) のような，双方向の文脈を必要とする下流課題のための強力な基盤を作ります。
もし，マスク化言語モデル課題でモデルを微調整したい場合は [run_mlm.py](https://github.com/huggingface/transformers/tree/master/examples/pytorch/language-modeling/run_mlm.py) スクリプトを活用することができます。
<!-- Masked language modeling is the task of masking tokens in a sequence with a masking token, and prompting the model to fill that mask with an appropriate token. 
This allows the model to attend to both the right context (tokens on the right of the mask) and the left context (tokens on the left of the mask). 
Such a training creates a strong basis for downstream tasks requiring bi-directional context, such as SQuAD (question answering, see [Lewis, Lui, Goyal et al.](https://arxiv.org/abs/1910.13461), part 4.2). 
If you would like to fine-tune a model on a masked language modeling task, you may leverage the [run_mlm.py](https://github.com/huggingface/transformers/tree/master/examples/pytorch/language-modeling/run_mlm.py) script.-->

ここでは，パイプラインを使って系列からマスクを置き換える例を紹介します。
<!-- Here is an example of using pipelines to replace a mask from a sequence: -->

In [ ]:
from transformers import pipeline
unmasker = pipeline("fill-mask")

これにより，マスクが満たされた配列，信頼性得点，トークン化された語彙のトークンID が出力されます。
<!-- This outputs the sequences with the mask filled, the confidence score, and the token id in the tokenizer vocabulary: -->

In [ ]:
from pprint import pprint

pprint(
    unmasker(
        f"HuggingFace is creating a {unmasker.tokenizer.mask_token} that the community uses to solve NLP tasks."
    )
)

[{'score': 0.17927499115467072,
  'sequence': 'HuggingFace is creating a tool that the community uses to solve '
              'NLP tasks.',
  'token': 3944,
  'token_str': ' tool'},
 {'score': 0.11349404603242874,
  'sequence': 'HuggingFace is creating a framework that the community uses to '
              'solve NLP tasks.',
  'token': 7208,
  'token_str': ' framework'},
 {'score': 0.05243541672825813,
  'sequence': 'HuggingFace is creating a library that the community uses to '
              'solve NLP tasks.',
  'token': 5560,
  'token_str': ' library'},
 {'score': 0.034935448318719864,
  'sequence': 'HuggingFace is creating a database that the community uses to '
              'solve NLP tasks.',
  'token': 8503,
  'token_str': ' database'},
 {'score': 0.028602484613656998,
  'sequence': 'HuggingFace is creating a prototype that the community uses to '
              'solve NLP tasks.',
  'token': 17715,
  'token_str': ' prototype'}]


In [ ]:
from transformers import pipeline

model_name_ja = 'cl-tohoku/bert-base-japanese'
unmasker_ja = pipeline("fill-mask", model=model_name_ja, tokenizer=model_name_ja)

from pprint import pprint

pprint(
    unmasker_ja(
        f"月日は百代の過客にして、行き交ふ年もまた {unmasker_ja.tokenizer.mask_token} なり。."
    )
)

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'score': 0.04446588456630707,
  'sequence': '月日 は 百 代 の 過客 に し て 、 行き 交 ふ 年 も また なり 。.',
  'token': 1,
  'token_str': '[ U N K ]'},
 {'score': 0.036625366657972336,
  'sequence': '月日 は 百 代 の 過客 に し て 、 行き 交 ふ 年 も また 同じ なり 。.',
  'token': 552,
  'token_str': '同 じ'},
 {'score': 0.02357492409646511,
  'sequence': '月日 は 百 代 の 過客 に し て 、 行き 交 ふ 年 も また 長く なり 。.',
  'token': 4109,
  'token_str': '長 く'},
 {'score': 0.019527362659573555,
  'sequence': '月日 は 百 代 の 過客 に し て 、 行き 交 ふ 年 も また 正月 なり 。.',
  'token': 9782,
  'token_str': '正 月'},
 {'score': 0.01736573502421379,
  'sequence': '月日 は 百 代 の 過客 に し て 、 行き 交 ふ 年 も また 毎年 なり 。.',
  'token': 2617,
  'token_str': '毎 年'}]


ここでは，モデルとトークン化器を使って，マスク化言語モデルを行う例を紹介します。
処理は以下の通りです。
<!-- Here is an example of doing masked language modeling using a model and a tokenizer. 
The process is the following: -->

1. チェックポイント名からトークン化器とモデルを実体化します。モデルは DistilBERT モデルとして識別され，チェックポイントに保存されている重みでロードされます
2. 単語の代わりに `tokenizer.mask_token` を置いて，マスクトークンを持つ系列を定義します
3. そのシーケンスを ID のリストに符号化し，そのリスト内のマスクトークンの位置を見つける
4. このテンソルは，語彙と同じサイズで，値は各トークンに割り当てられた得点です
モデルは，その文脈で可能性が高いと判断したトークンに高い得点を与えます
5. PyTorchの `topk` または TensorFlow の `top_k` メソッドを使用して，上位 5 つのトークンを取得します
6. マスクトークンをトークンで置き換えて，結果を表示します

<!-- 
1. Instantiate a tokenizer and a model from the checkpoint name. The model is identified as a DistilBERT model and loads it with the weights stored in the checkpoint.
2. Define a sequence with a masked token, placing the `tokenizer.mask_token` instead of a word. 
3. Encode that sequence into a list of IDs and find the position of the masked token in that list.
4. Retrieve the predictions at the index of the mask token: this tensor has the same size as the vocabulary, and the values are the scores attributed to each token. 
The model gives higher score to tokens it deems probable in that context.
5. Retrieve the top 5 tokens using the PyTorch `topk` or TensorFlow `top_k` methods.
6. Replace the mask token by the tokens and print the results -->

In [15]:
from transformers import AutoModelForMaskedLM, AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-cased")
model = AutoModelForMaskedLM.from_pretrained("distilbert-base-cased")

sequence = (
    "Distilled models are smaller than the models they mimic. Using them instead of the large "
    f"versions would help {tokenizer.mask_token} our carbon footprint."
)

inputs = tokenizer(sequence, return_tensors="pt")
mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]

token_logits = model(**inputs).logits
mask_token_logits = token_logits[0, mask_token_index, :]

top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()

for token in top_5_tokens:
    print(sequence.replace(tokenizer.mask_token, tokenizer.decode([token])))

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/411 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/251M [00:00<?, ?B/s]

Distilled models are smaller than the models they mimic. Using them instead of the large versions would help reduce our carbon footprint.
Distilled models are smaller than the models they mimic. Using them instead of the large versions would help increase our carbon footprint.
Distilled models are smaller than the models they mimic. Using them instead of the large versions would help decrease our carbon footprint.
Distilled models are smaller than the models they mimic. Using them instead of the large versions would help offset our carbon footprint.
Distilled models are smaller than the models they mimic. Using them instead of the large versions would help improve our carbon footprint.


In [23]:
# 直上セルの日本語版
from transformers import AutoModelForMaskedLM, AutoTokenizer
import torch

model_name_ja = 'cl-tohoku/bert-base-japanese'
tknz = AutoTokenizer.from_pretrained(model_name_ja)
model_ja = AutoModelForMaskedLM.from_pretrained(model_name_ja)

sequence_ja = (
    "吾輩は猫である。" f"名前はまだ{tknz_ja.mask_token}。"
)

inputs_ja = tknz(sequence_ja, truncation=True, padding=True, return_tensors="pt")
mask_token_index_ja = torch.where(inputs_ja["input_ids"] == tknz.mask_token_id)[1]

token_logits_ja = model_ja(**inputs_ja).logits
mask_token_logits_ja = token_logits_ja[0, mask_token_index_ja, :]

top_5_tokens_ja = torch.topk(mask_token_logits_ja, 5, dim=1).indices[0].tolist()

for token in top_5_tokens_ja:
    print(sequence_ja.replace(tknz.mask_token, tknz.decode([token])))

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


吾輩は猫である。名前はまだ不明。
吾輩は猫である。名前はまだない。
吾輩は猫である。名前はまだ赤ちゃん。
吾輩は猫である。名前はまだら。
吾輩は猫である。名前はまだ若い。


In [24]:
# Tensorflow 版なのでコメントアウト
# from transformers import TFAutoModelForMaskedLM, AutoTokenizer
# import tensorflow as tf

# tokenizer = AutoTokenizer.from_pretrained("distilbert-base-cased")
# model = TFAutoModelForMaskedLM.from_pretrained("distilbert-base-cased")

# sequence = (
#     "Distilled models are smaller than the models they mimic. Using them instead of the large "
#     f"versions would help {tokenizer.mask_token} our carbon footprint."
# )

# inputs = tokenizer(sequence, return_tensors="tf")
# mask_token_index = tf.where(inputs["input_ids"] == tokenizer.mask_token_id)[0, 1]

# token_logits = model(**inputs).logits
# mask_token_logits = token_logits[0, mask_token_index, :]

# top_5_tokens = tf.math.top_k(mask_token_logits, 5).indices.numpy()

# for token in top_5_tokens:
#     print(sequence.replace(tokenizer.mask_token, tokenizer.decode([token])))

これにより，モデルが予測した上位 5 つのトークンを含む 5 つの系列が表示されます。
<!-- This prints five sequences, with the top 5 tokens predicted by the model. -->

### 3.2 因果言語モデル
<!-- ### Causal Language Modeling -->

因果言語モデルとは，一連のトークンの後に続くトークンを予測する課題です。
この状況では，モデルは左の文脈 (マスクの左にあるトークン) にのみ注意を払います。
このような訓練は，特に文生成課題において興味深いものです。
因果言語モデル課題でモデルを微調整したい場合は，[run_clm.py](https://github.com/huggingface/transformers/tree/master/examples/pytorch/language-modeling/run_clm.py) スクリプトを活用できます。
<!-- Causal language modeling is the task of predicting the token following a sequence of tokens. 
In this situation, the model only attends to the left context (tokens on the left of the mask). 
Such a training is particularly interesting for generation tasks. 
If you would like to fine-tune a model on a causal language modeling task, you may leverage the [run_clm.py](https://github.com/huggingface/transformers/tree/master/examples/pytorch/language-modeling/run_clm.py) script.-->

通常，次のトークンは，モデルが入力系列から生成する最後の隠れ状態のロジット比からサンプリングして予測します。
<!-- Usually, the next token is predicted by sampling from the logits of the last hidden state the model produces from the input sequence. -->

ここでは，トークン化器とモデルを使い `PreTrainedModel.top_k_top_p_filtering` メソッドを活用して，トークンの入力系列に続く次のトークンをサンプリングする例を紹介します。
<!-- Here is an example of using the tokenizer and model and leveraging the `PreTrainedModel.top_k_top_p_filtering` method to sample the next token following an input sequence of tokens. -->

In [28]:
from transformers import AutoModelForCausalLM, AutoTokenizer, top_k_top_p_filtering
import torch
from torch import nn


tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2")

sequence = f"Hugging Face is based in DUMBO, New York City, and"

inputs = tokenizer(sequence, return_tensors="pt")
input_ids = inputs["input_ids"]

# get logits of last hidden state
next_token_logits = model(**inputs).logits[:, -1, :]

# filter
filtered_next_token_logits = top_k_top_p_filtering(next_token_logits, top_k=50, top_p=1.0)

# sample
probs = nn.functional.softmax(filtered_next_token_logits, dim=-1)
next_token = torch.multinomial(probs, num_samples=1)

generated = torch.cat([input_ids, next_token], dim=-1)

resulting_string = tokenizer.decode(generated.tolist()[0])
print(resulting_string)

Hugging Face is based in DUMBO, New York City, and features


In [31]:
# 直上セルの日本語版
from transformers import AutoModelForCausalLM, AutoTokenizer, top_k_top_p_filtering
import torch
from torch import nn

model_name_ja = 'cl-tohoku/bert-base-japanese'
tknz = AutoTokenizer.from_pretrained(model_name_ja)
model_ja = AutoModelForCausalLM.from_pretrained(model_name_ja, is_decoder=True)

sequence_ja = f"月日は百代の過客にして、行かふ年も又旅人"

inputs_ja = tknz(sequence_ja, truncation=True, padding=True, return_tensors="pt")
input_ids_ja = inputs_ja["input_ids"]

# 最上位中間層の確率比を取得する # get logits of last hidden state
next_token_logits_ja = model(**inputs_ja).logits[:, -1, :]

# フィルタ filter
filtered_next_token_logits_ja = top_k_top_p_filtering(next_token_logits_ja, top_k=50, top_p=1.0)

# サンプリング sample
probs_ja = nn.functional.softmax(filtered_next_token_logits_ja, dim=-1)
next_token_ja = torch.multinomial(probs_ja, num_samples=1)

generated_ja = torch.cat([input_ids_ja, next_token_ja], dim=-1)

resulting_string_ja = tknz.decode(generated_ja.tolist()[0])
print(resulting_string_ja)

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese were not used when initializing BertLMHeadModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertLMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertLMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[CLS] 月日 は 百 代 の 過客 に し て 、 行か ふ 年 も 又 旅人 [SEP] [PAD]


In [32]:
# Tensorflow 版なのでコメントアウト
# from transformers import TFAutoModelForCausalLM, AutoTokenizer, tf_top_k_top_p_filtering
# import tensorflow as tf

# tokenizer = AutoTokenizer.from_pretrained("gpt2")
# model = TFAutoModelForCausalLM.from_pretrained("gpt2")

# sequence = f"Hugging Face is based in DUMBO, New York City, and"

# inputs = tokenizer(sequence, return_tensors="tf")
# input_ids = inputs["input_ids"]

# # get logits of last hidden state
# next_token_logits = model(**inputs).logits[:, -1, :]

# # filter
# filtered_next_token_logits = tf_top_k_top_p_filtering(next_token_logits, top_k=50, top_p=1.0)

# # sample
# next_token = tf.random.categorical(filtered_next_token_logits, dtype=tf.int32, num_samples=1)

# generated = tf.concat([input_ids, next_token], axis=1)

# resulting_string = tokenizer.decode(generated.numpy().tolist()[0])
# print(resulting_string)

これにより，元の配列に続く (うまくいけば) まとまった次のトークンが出力されます。
この例では，単語の  *is*  や  *features*  が出力されます。
<!-- This outputs a (hopefully) coherent next token following the original sequence, which in our case is the word *is* or *features*.-->

次節では [generation_utils.GenerationMixin.generate()](https://huggingface.co/docs/transformers/master/en/main_classes/model#transformers.generation_utils.GenerationMixin.generate) を使って，一度に一つのトークンではなく，指定した長さまでの複数のトークンを生成する方法を紹介します。
<!-- In the next section, we show how [generation_utils.GenerationMixin.generate()](https://huggingface.co/docs/transformers/master/en/main_classes/model#transformers.generation_utils.GenerationMixin.generate) can be used to generate multiple tokens up to a specified length instead of one token at a time. -->

### 3.3 文生成
<!-- ### Text Generation -->

テキスト生成 (*a.k.a* *open-ended text generation*) では，与えられた文脈から続く，まとまった部分のテキストを作成することが目的です。
次の例では，パイプラインでテキストを生成するために，*GPT-2* をどのように使用するかを示しています。
デフォルトでは，すべてのモデルは，パイプラインで使用されるときに，それぞれの設定 (例として [gpt-2 config](https://huggingface.co/gpt2/blob/main/config.json) を参照) に応じて，*Top-K* サンプリングを適用します。
<!-- In text generation (*a.k.a* *open-ended text generation*) the goal is to create a coherent portion of text that is a continuation from the given context. 
The following example shows how *GPT-2* can be used in pipelines to generate text.
As a default all models apply *Top-K* sampling when used in pipelines, as configured in their respective configurations (see [gpt-2 config](https://huggingface.co/gpt2/blob/main/config.json) for example). -->

In [33]:
from transformers import pipeline

text_generator = pipeline("text-generation")
print(text_generator("As far as I am concerned, I will", max_length=50, do_sample=False))

No model was supplied, defaulted to gpt2 (https://huggingface.co/gpt2)
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'As far as I am concerned, I will be the first to admit that I am not a fan of the idea of a "free market." I think that the idea of a free market is a bit of a stretch. I think that the idea'}]


In [38]:
#直上セルの日本語版
from transformers import pipeline

model_name_ja = 'cl-tohoku/bert-base-japanese'
text_generator_ja = pipeline("text-generation", model=model_name_ja, tokenizer=model_name_ja, is_decoder=True)

print(text_generator_ja("私の知る限り鬼滅の刃の視聴率は，", max_length=50, do_sample=False, is_decoder=True))
print(text_generator_ja("明日の天気は", max_length=50, do_sample=False, is_decoder=True))

If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`
Some weights of the model checkpoint at cl-tohoku/bert-base-japanese were not used when initializing BertLMHeadModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertLMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertLMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'generated_text': '私の知る限り鬼滅の刃の視聴率は，,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,'}]
[{'generated_text': '明日の天気は は は は は は は は は は は は は は は は は は は は は は は は は は は は は は は は は は は は は は は は は は は は は は は'}]


ここでは，モデルはコンテキスト  *"As far as I am concerned, I will “*  から，トークンの合計最大長が *50* のランダムなテキストを生成します。
舞台裏では，パイプラインオブジェクトが [PreTrainedModel.generate()](https://huggingface.co/docs/transformers/master/en/main_classes/model#transformers.generation_utils.GenerationMixin.generate) というメソッドを呼び出し，テキストを生成しています。
このメソッドのデフォルトの引数は `max_length` と `do_sample` のように，パイプラインで上書きすることができます。
<!-- Here, the model generates a random text with a total maximal length of *50* tokens from context *"As far as I am concerned, I will"*. Behind the scenes, the pipeline object calls the method [PreTrainedModel.generate()](https://huggingface.co/docs/transformers/master/en/main_classes/model#transformers.generation_utils.GenerationMixin.generate) to generate text. The default arguments for this method can be
overridden in the pipeline, as is shown above for the arguments `max_length` and `do_sample`.-->

以下は `XLNet` とそのトークン化器を使ったテキスト生成例で `generate()` を直接呼び出しています。
<!-- Below is an example of text generation using `XLNet` and its tokenizer, which includes calling `generate()` directly: -->

In [39]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("xlnet-base-cased")
tokenizer = AutoTokenizer.from_pretrained("xlnet-base-cased")

# Padding text helps XLNet with short prompts - proposed by Aman Rusia in https://github.com/rusiaaman/XLNet-gen#methodology
PADDING_TEXT = """In 1991, the remains of Russian Tsar Nicholas II and his family
(except for Alexei and Maria) are discovered.
The voice of Nicholas's young son, Tsarevich Alexei Nikolaevich, narrates the
remainder of the story. 1883 Western Siberia,
a young Grigori Rasputin is asked by his father and a group of men to perform magic.
Rasputin has a vision and denounces one of the men as a horse thief. Although his
father initially slaps him for making such an accusation, Rasputin watches as the
man is chased outside and beaten. Twenty years later, Rasputin sees a vision of
the Virgin Mary, prompting him to become a priest. Rasputin quickly becomes famous,
with people, even a bishop, begging for his blessing. <eod> </s> <eos>"""

prompt = "Today the weather is really nice and I am planning on "
inputs = tokenizer(PADDING_TEXT + prompt, add_special_tokens=False, return_tensors="pt")["input_ids"]

prompt_length = len(tokenizer.decode(inputs[0]))
outputs = model.generate(inputs, max_length=250, do_sample=True, top_p=0.95, top_k=60)
generated = prompt + tokenizer.decode(outputs[0])[prompt_length + 1 :]

print(generated)

Downloading:   0%|          | 0.00/760 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/445M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/779k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

Today the weather is really nice and I am planning on moving into the house. I'll be moving in the fall or spring in the fall. I'll be renting some of the space and having the first three of four bedrooms for each house.<eop> About 4 - 5 PM. I should be having a very nice day off. The first half of my day off, about six hours on a Sunday, at


In [43]:
#直上セルの日本語版
from transformers import AutoModelForCausalLM, AutoTokenizer


model_name_ja = 'cl-tohoku/bert-base-japanese'
model_ja = AutoModelForCausalLM.from_pretrained(model_name_ja, is_decoder=True)
tknz = AutoTokenizer.from_pretrained(model_name_ja)

# Padding text helps XLNet with short prompts - proposed by Aman Rusia in https://github.com/rusiaaman/XLNet-gen#methodology
PADDING_TEXT_ja = """1991 年、ロシア皇帝ニコライ 2 世とその家族（アレクセイとマリアを除く）の遺骨が発見された。
ニコライの幼い息子、アレクセイ・ニコラエヴィッチの声で、物語の続きが語られる。
1883 年 西シベリア，若き日のグリゴリー・ラスプーチンは、父と男たちに魔法をかけてほしいと頼まれる。
ラスプーチンは幻覚を見て、男たちの一人を馬泥棒として糾弾する。
父はそんな告発をした彼を平手打ちしたが、ラスプーチンはその男が外に追い出されて殴られるのを見ていた。
20 年後、ラスプーチンは聖母マリアの幻影を見て、司祭になることを決意する。
ラスプーチンは瞬く間に有名になり、人々や司教までもが彼の祝福を求めるようになる。"""

prompt_ja = "今日はとてもいい天気なので、私は次のように計画しています。"
inputs_ja = tknz(PADDING_TEXT_ja + prompt_ja, add_special_tokens=False, return_tensors="pt")["input_ids"]

prompt_length_ja = len(tknz.decode(inputs_ja[0]))
outputs_ja = model_ja.generate(inputs_ja, max_length=250, do_sample=True, top_p=0.95, top_k=60)
generated_ja = prompt_ja + tknz.decode(outputs_ja[0])[prompt_length_ja + 1 :]

print(generated_ja)

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese were not used when initializing BertLMHeadModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertLMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertLMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


今日はとてもいい天気なので、私は次のように計画しています。。 。 。 。 」 」 」 」 」 」 」 」 」 」 」 」 「 「 「 「 「 「 」 」 」 」 」 」 」 」 」 」 」 」 」 」 「 「 「 「 「


In [ ]:
# Tensorflow 版なのでコメントアウト

# from transformers import TFAutoModelForCausalLM, AutoTokenizer

# model = TFAutoModelForCausalLM.from_pretrained("xlnet-base-cased")
# tokenizer = AutoTokenizer.from_pretrained("xlnet-base-cased")

# # Padding text helps XLNet with short prompts - proposed by Aman Rusia in https://github.com/rusiaaman/XLNet-gen#methodology
# PADDING_TEXT = """In 1991, the remains of Russian Tsar Nicholas II and his family
# (except for Alexei and Maria) are discovered.
# The voice of Nicholas's young son, Tsarevich Alexei Nikolaevich, narrates the
# remainder of the story. 1883 Western Siberia,
# a young Grigori Rasputin is asked by his father and a group of men to perform magic.
# Rasputin has a vision and denounces one of the men as a horse thief. Although his
# father initially slaps him for making such an accusation, Rasputin watches as the
# man is chased outside and beaten. Twenty years later, Rasputin sees a vision of
# the Virgin Mary, prompting him to become a priest. Rasputin quickly becomes famous,
# with people, even a bishop, begging for his blessing. <eod> </s> <eos>"""

# prompt = "Today the weather is really nice and I am planning on "
# inputs = tokenizer(PADDING_TEXT + prompt, add_special_tokens=False, return_tensors="tf")["input_ids"]

# prompt_length = len(tokenizer.decode(inputs[0]))
# outputs = model.generate(inputs, max_length=250, do_sample=True, top_p=0.95, top_k=60)
# generated = prompt + tokenizer.decode(outputs[0])[prompt_length + 1 :]

# print(generated)

Today the weather is really nice and I am planning ...

テキスト生成は現在，PyTorch の *GPT-2*，*OpenAi-GPT*，*CTRL*，*XLNet*，*Transfo-XL*，*Reformer* で可能で，Tensorflow でもほとんどのモデルで可能です。
上の例でわかるように，*XLNet* と *Transfo-XL* は，うまく動作させるためにしばしばパディングが必要です。
GPT-2 は，因果言語モデルを目的とした何百万ものウェブページで訓練されているため，通常は *オープンエンドのテキスト生成* に適しています。

テキスト生成に様々なデコードストラテジーを適用する方法については，テキスト生成のブログ記事 [こちら](https://huggingface.co/blog/how-to-generate) もご参照ください。
<!-- Text generation is currently possible with *GPT-2*, *OpenAi-GPT*, *CTRL*, *XLNet*, *Transfo-XL* and *Reformer* in PyTorch and for most models in Tensorflow as well. 
As can be seen in the example above *XLNet* and *Transfo-XL* often need to be padded to work well. GPT-2 is usually a good choice for *open-ended text generation* because it was trained on millions of webpages with a causal language modeling objective.

For more information on how to apply different decoding strategies for text generation, please also refer to our text generation blog post [here](https://huggingface.co/blog/how-to-generate). -->

## Named Entity Recognition

Named Entity Recognition (NER) is the task of classifying tokens according to a class, for example, identifying a token
as a person, an organisation or a location. An example of a named entity recognition dataset is the CoNLL-2003 dataset,
which is entirely based on that task. If you would like to fine-tune a model on an NER task, you may leverage the
[run_ner.py](https://github.com/huggingface/transformers/tree/master/examples/pytorch/token-classification/run_ner.py) script.

Here is an example of using pipelines to do named entity recognition, specifically, trying to identify tokens as
belonging to one of 9 classes:

- O, Outside of a named entity
- B-MIS, Beginning of a miscellaneous entity right after another miscellaneous entity
- I-MIS, Miscellaneous entity
- B-PER, Beginning of a person's name right after another person's name
- I-PER, Person's name
- B-ORG, Beginning of an organisation right after another organisation
- I-ORG, Organisation
- B-LOC, Beginning of a location right after another location
- I-LOC, Location

It leverages a fine-tuned model on CoNLL-2003, fine-tuned by [@stefan-it](https://github.com/stefan-it) from [dbmdz](https://github.com/dbmdz).

In [ ]:
from transformers import pipeline

ner_pipe = pipeline("ner")

sequence = """Hugging Face Inc. is a company based in New York City. Its headquarters are in DUMBO,
therefore very close to the Manhattan Bridge which is visible from the window."""

This outputs a list of all words that have been identified as one of the entities from the 9 classes defined above.
Here are the expected results:

In [ ]:
for entity in ner_pipe(sequence):
    print(entity)

{'entity': 'I-ORG', 'score': 0.9996, 'index': 1, 'word': 'Hu', 'start': 0, 'end': 2}
{'entity': 'I-ORG', 'score': 0.9910, 'index': 2, 'word': '##gging', 'start': 2, 'end': 7}
{'entity': 'I-ORG', 'score': 0.9982, 'index': 3, 'word': 'Face', 'start': 8, 'end': 12}
{'entity': 'I-ORG', 'score': 0.9995, 'index': 4, 'word': 'Inc', 'start': 13, 'end': 16}
{'entity': 'I-LOC', 'score': 0.9994, 'index': 11, 'word': 'New', 'start': 40, 'end': 43}
{'entity': 'I-LOC', 'score': 0.9993, 'index': 12, 'word': 'York', 'start': 44, 'end': 48}
{'entity': 'I-LOC', 'score': 0.9994, 'index': 13, 'word': 'City', 'start': 49, 'end': 53}
{'entity': 'I-LOC', 'score': 0.9863, 'index': 19, 'word': 'D', 'start': 79, 'end': 80}
{'entity': 'I-LOC', 'score': 0.9514, 'index': 20, 'word': '##UM', 'start': 80, 'end': 82}
{'entity': 'I-LOC', 'score': 0.9337, 'index': 21, 'word': '##BO', 'start': 82, 'end': 84}
{'entity': 'I-LOC', 'score': 0.9762, 'index': 28, 'word': 'Manhattan', 'start': 114, 'end': 123}
{'entity': 'I-LO

Note how the tokens of the sequence "Hugging Face" have been identified as an organisation, and "New York City",
"DUMBO" and "Manhattan Bridge" have been identified as locations.

Here is an example of doing named entity recognition, using a model and a tokenizer. The process is the following:

1. Instantiate a tokenizer and a model from the checkpoint name. The model is identified as a BERT model and loads it
   with the weights stored in the checkpoint.
2. Define a sequence with known entities, such as "Hugging Face" as an organisation and "New York City" as a location.
3. Split words into tokens so that they can be mapped to predictions. We use a small hack by, first, completely
   encoding and decoding the sequence, so that we're left with a string that contains the special tokens.
4. Encode that sequence into IDs (special tokens are added automatically).
5. Retrieve the predictions by passing the input to the model and getting the first output. This results in a
   distribution over the 9 possible classes for each token. We take the argmax to retrieve the most likely class for
   each token.
6. Zip together each token with its prediction and print it.

In [ ]:
from transformers import AutoModelForTokenClassification, AutoTokenizer
import torch

model = AutoModelForTokenClassification.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

sequence = (
    "Hugging Face Inc. is a company based in New York City. Its headquarters are in DUMBO, "
    "therefore very close to the Manhattan Bridge."
)

inputs = tokenizer(sequence, return_tensors="pt")
tokens = inputs.tokens()

outputs = model(**inputs).logits
predictions = torch.argmax(outputs, dim=2)

In [ ]:
from transformers import TFAutoModelForTokenClassification, AutoTokenizer
import tensorflow as tf

model = TFAutoModelForTokenClassification.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

sequence = (
    "Hugging Face Inc. is a company based in New York City. Its headquarters are in DUMBO, "
    "therefore very close to the Manhattan Bridge."
)

inputs = tokenizer(sequence, return_tensors="tf")
tokens = inputs.tokens()

outputs = model(**inputs)[0]
predictions = tf.argmax(outputs, axis=2)

This outputs a list of each token mapped to its corresponding prediction. Differently from the pipeline, here every
token has a prediction as we didn't remove the "0"th class, which means that no particular entity was found on that
token.

In the above example, `predictions` is an integer that corresponds to the predicted class. We can use the
`model.config.id2label` property in order to recover the class name corresponding to the class number, which is
illustrated below:

In [ ]:
for token, prediction in zip(tokens, predictions[0].numpy()):
    print((token, model.config.id2label[prediction]))

('[CLS]', 'O')
('Hu', 'I-ORG')
('##gging', 'I-ORG')
('Face', 'I-ORG')
('Inc', 'I-ORG')
('.', 'O')
('is', 'O')
('a', 'O')
('company', 'O')
('based', 'O')
('in', 'O')
('New', 'I-LOC')
('York', 'I-LOC')
('City', 'I-LOC')
('.', 'O')
('Its', 'O')
('headquarters', 'O')
('are', 'O')
('in', 'O')
('D', 'I-LOC')
('##UM', 'I-LOC')
('##BO', 'I-LOC')
(',', 'O')
('therefore', 'O')
('very', 'O')
('close', 'O')
('to', 'O')
('the', 'O')
('Manhattan', 'I-LOC')
('Bridge', 'I-LOC')
('.', 'O')
('[SEP]', 'O')

## Summarization

Summarization is the task of summarizing a document or an article into a shorter text. If you would like to fine-tune a
model on a summarization task, you may leverage the [run_summarization.py](https://github.com/huggingface/transformers/tree/master/examples/pytorch/summarization/run_summarization.py)
script.

An example of a summarization dataset is the CNN / Daily Mail dataset, which consists of long news articles and was
created for the task of summarization. If you would like to fine-tune a model on a summarization task, various
approaches are described in this [document](https://github.com/huggingface/transformers/tree/master/examples/pytorch/summarization/README.md).

Here is an example of using the pipelines to do summarization. It leverages a Bart model that was fine-tuned on the CNN
/ Daily Mail data set.

In [ ]:
from transformers import pipeline

summarizer = pipeline("summarization")

ARTICLE = """ New York (CNN)When Liana Barrientos was 23 years old, she got married in Westchester County, New York.
A year later, she got married again in Westchester County, but to a different man and without divorcing her first husband.
Only 18 days after that marriage, she got hitched yet again. Then, Barrientos declared "I do" five more times, sometimes only within two weeks of each other.
In 2010, she married once more, this time in the Bronx. In an application for a marriage license, she stated it was her "first and only" marriage.
Barrientos, now 39, is facing two criminal counts of "offering a false instrument for filing in the first degree," referring to her false statements on the
2010 marriage license application, according to court documents.
Prosecutors said the marriages were part of an immigration scam.
On Friday, she pleaded not guilty at State Supreme Court in the Bronx, according to her attorney, Christopher Wright, who declined to comment further.
After leaving court, Barrientos was arrested and charged with theft of service and criminal trespass for allegedly sneaking into the New York subway through an emergency exit, said Detective
Annette Markowski, a police spokeswoman. In total, Barrientos has been married 10 times, with nine of her marriages occurring between 1999 and 2002.
All occurred either in Westchester County, Long Island, New Jersey or the Bronx. She is believed to still be married to four men, and at one time, she was married to eight men at once, prosecutors say.
Prosecutors said the immigration scam involved some of her husbands, who filed for permanent residence status shortly after the marriages.
Any divorces happened only after such filings were approved. It was unclear whether any of the men will be prosecuted.
The case was referred to the Bronx District Attorney\'s Office by Immigration and Customs Enforcement and the Department of Homeland Security\'s
Investigation Division. Seven of the men are from so-called "red-flagged" countries, including Egypt, Turkey, Georgia, Pakistan and Mali.
Her eighth husband, Rashid Rajput, was deported in 2006 to his native Pakistan after an investigation by the Joint Terrorism Task Force.
If convicted, Barrientos faces up to four years in prison.  Her next court appearance is scheduled for May 18.
"""

Because the summarization pipeline depends on the `PreTrainedModel.generate()` method, we can override the default
arguments of `PreTrainedModel.generate()` directly in the pipeline for `max_length` and `min_length` as shown
below. This outputs the following summary:

In [ ]:
print(summarizer(ARTICLE, max_length=130, min_length=30, do_sample=False))

[{'summary_text': ' Liana Barrientos, 39, is charged with two counts of "offering a false instrument for filing in
the first degree" In total, she has been married 10 times, with nine of her marriages occurring between 1999 and
2002 . At one time, she was married to eight men at once, prosecutors say .'}]

Here is an example of doing summarization using a model and a tokenizer. The process is the following:

1. Instantiate a tokenizer and a model from the checkpoint name. Summarization is usually done using an encoder-decoder
   model, such as `Bart` or `T5`.
2. Define the article that should be summarized.
3. Add the T5 specific prefix "summarize: ".
4. Use the `PreTrainedModel.generate()` method to generate the summary.

In this example we use Google's T5 model. Even though it was pre-trained only on a multi-task mixed dataset (including
CNN / Daily Mail), it yields very good results.

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model = AutoModelForSeq2SeqLM.from_pretrained("t5-base")
tokenizer = AutoTokenizer.from_pretrained("t5-base")

# T5 uses a max_length of 512 so we cut the article to 512 tokens.
inputs = tokenizer("summarize: " + ARTICLE, return_tensors="pt", max_length=512, truncation=True)
outputs = model.generate(
    inputs["input_ids"], max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True
)

print(tokenizer.decode(outputs[0]))

<pad> prosecutors say the marriages were part of an immigration scam. if convicted, barrientos faces two criminal
counts of "offering a false instrument for filing in the first degree" she has been married 10 times, nine of them
between 1999 and 2002.</s>

In [ ]:
from transformers import TFAutoModelForSeq2SeqLM, AutoTokenizer

model = TFAutoModelForSeq2SeqLM.from_pretrained("t5-base")
tokenizer = AutoTokenizer.from_pretrained("t5-base")

# T5 uses a max_length of 512 so we cut the article to 512 tokens.
inputs = tokenizer("summarize: " + ARTICLE, return_tensors="tf", max_length=512)
outputs = model.generate(
    inputs["input_ids"], max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True
)

print(tokenizer.decode(outputs[0]))

<pad> prosecutors say the marriages were part of an immigration scam. if convicted, barrientos faces two criminal
counts of "offering a false instrument for filing in the first degree" she has been married 10 times, nine of them
between 1999 and 2002.

## Translation

Translation is the task of translating a text from one language to another. If you would like to fine-tune a model on a
translation task, you may leverage the [run_translation.py](https://github.com/huggingface/transformers/tree/master/examples/pytorch/translation/run_translation.py) script.

An example of a translation dataset is the WMT English to German dataset, which has sentences in English as the input
data and the corresponding sentences in German as the target data. If you would like to fine-tune a model on a
translation task, various approaches are described in this [document](https://github.com/huggingface/transformers/tree/master/examples/pytorch/translation/README.md).

Here is an example of using the pipelines to do translation. It leverages a T5 model that was only pre-trained on a
multi-task mixture dataset (including WMT), yet, yielding impressive translation results.

In [ ]:
from transformers import pipeline

translator = pipeline("translation_en_to_de")
print(translator("Hugging Face is a technology company based in New York and Paris", max_length=40))

[{'translation_text': 'Hugging Face ist ein Technologieunternehmen mit Sitz in New York und Paris.'}]

Because the translation pipeline depends on the `PreTrainedModel.generate()` method, we can override the default
arguments of `PreTrainedModel.generate()` directly in the pipeline as is shown for `max_length` above.

Here is an example of doing translation using a model and a tokenizer. The process is the following:

1. Instantiate a tokenizer and a model from the checkpoint name. Summarization is usually done using an encoder-decoder
   model, such as `Bart` or `T5`.
2. Define the article that should be summarized.
3. Add the T5 specific prefix "translate English to German: "
4. Use the `PreTrainedModel.generate()` method to perform the translation.

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model = AutoModelForSeq2SeqLM.from_pretrained("t5-base")
tokenizer = AutoTokenizer.from_pretrained("t5-base")

inputs = tokenizer(
    "translate English to German: Hugging Face is a technology company based in New York and Paris",
    return_tensors="pt",
)
outputs = model.generate(inputs["input_ids"], max_length=40, num_beams=4, early_stopping=True)

print(tokenizer.decode(outputs[0]))

<pad> Hugging Face ist ein Technologieunternehmen mit Sitz in New York und Paris.</s>

In [ ]:
from transformers import TFAutoModelForSeq2SeqLM, AutoTokenizer

model = TFAutoModelForSeq2SeqLM.from_pretrained("t5-base")
tokenizer = AutoTokenizer.from_pretrained("t5-base")

inputs = tokenizer(
    "translate English to German: Hugging Face is a technology company based in New York and Paris",
    return_tensors="tf",
)
outputs = model.generate(inputs["input_ids"], max_length=40, num_beams=4, early_stopping=True)

print(tokenizer.decode(outputs[0]))

<pad> Hugging Face ist ein Technologieunternehmen mit Sitz in New York und Paris.

We get the same translation as with the pipeline example.